<a href="https://colab.research.google.com/github/Satwikram/Deep-Learning-Notebooks/blob/master/CNN/Fruit_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
"""
@Author: Satwik Ram K

"""
from google.colab import files

files.upload()


! mkdir ~/.kaggle


! cp kaggle.json ~/.kaggle/

! chmod 600 ~/.kaggle/kaggle.json


Saving kaggle.json to kaggle.json


In [3]:
!kaggle datasets download -d moltean/fruits

 99% 751M/760M [00:10<00:00, 138MB/s]
100% 760M/760M [00:10<00:00, 75.4MB/s]


In [ ]:
!unzip /content/fruits.zip

In [5]:
# Importing Dependecies
import numpy as np
import tensorflow as tf
from tensorflow import keras

In [7]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout, Activation
from tensorflow.keras.models import Sequential


In [8]:
train_data_dir = '/content/fruits-360/Training'
val_data_dir = '/content/fruits-360/Test'

In [56]:
# Data Augmentation
train_datagen = ImageDataGenerator(
     rescale=1./255,
     rotation_range=30,
     width_shift_range=0.3,
     height_shift_range=0.3,
     horizontal_flip=True,
     fill_mode='nearest'
)

In [57]:
validation_datagen = ImageDataGenerator(rescale=1./255)

In [58]:
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(32,32),
    batch_size=64,
    class_mode='categorical',
    shuffle=True
)

Found 67692 images belonging to 131 classes.


In [59]:
validation_generator = validation_datagen.flow_from_directory(
        val_data_dir,
        target_size=(32,32),
        batch_size=64,
        class_mode='categorical',
        shuffle=False)

Found 22688 images belonging to 131 classes.


In [70]:
#Defining the model
def create_model():
  model = Sequential()

  model.add(Conv2D(32, (3,3), input_shape = (32,32, 3), padding = 'same'))
  model.add(Activation('relu'))
  model.add(MaxPooling2D((2,2)))

  model.add(Conv2D(64, (3,3), padding = 'same'))
  model.add(Activation('relu'))
  model.add(MaxPooling2D((2,2)))
  model.add(Dropout(0.25))

  model.add(Conv2D(128, (3,3), padding = 'same'))
  model.add(Activation('relu'))
  model.add(MaxPooling2D((2,2)))
  model.add(Dropout(0.25))

  model.add(Conv2D(256, (3,3), padding = 'same'))
  model.add(Activation('relu'))
  model.add(MaxPooling2D((2,2)))
  model.add(Dropout(0.5))

  model.add(Flatten())
  model.add(Dense(512))
  model.add(Activation('relu'))
  model.add(Dropout(0.5))
  model.add(Dense(131))
  model.add(Activation('softmax'))

  return model



In [71]:
model = create_model()

In [72]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_16 (Conv2D)           (None, 32, 32, 32)        896       
_________________________________________________________________
activation_24 (Activation)   (None, 32, 32, 32)        0         
_________________________________________________________________
max_pooling2d_16 (MaxPooling (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 16, 16, 64)        18496     
_________________________________________________________________
activation_25 (Activation)   (None, 16, 16, 64)        0         
_________________________________________________________________
max_pooling2d_17 (MaxPooling (None, 8, 8, 64)          0         
_________________________________________________________________
dropout_16 (Dropout)         (None, 8, 8, 64)         

In [73]:
from tensorflow.keras.optimizers import SGD, RMSprop
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau


In [74]:
checkpoint = ModelCheckpoint("fruits.h5",
                             monitor="val_loss",
                             mode="min",
                             save_best_only = True,
                             verbose=1)

In [75]:
earlystop = EarlyStopping(monitor = 'val_loss', 
                          min_delta = 0, 
                          patience = 3,
                          verbose = 1,
                          restore_best_weights = True)

In [76]:
reduce_lr = ReduceLROnPlateau(monitor = 'val_loss',
                              factor = 0.2,
                              patience = 3,
                              verbose = 1,
                              min_delta = 0.0001)

In [77]:
# we put our call backs into a callback list
callbacks = [earlystop, checkpoint, reduce_lr]

In [78]:
# We use a very small learning rate 
model.compile(loss = 'categorical_crossentropy',
              optimizer = RMSprop(lr = 0.001),
              metrics = ['accuracy'])


In [79]:
model.fit_generator(train_generator, validation_data = validation_generator, epochs = 10, callbacks = callbacks)

Epoch 1/10
1058/1058 [==============================] - ETA: 0s - loss: 2.8058 - accuracy: 0.2341
Epoch 00001: val_loss improved from inf to 1.04322, saving model to fruits.h5
1058/1058 [==============================] - 270s 255ms/step - loss: 2.8058 - accuracy: 0.2341 - val_loss: 1.0432 - val_accuracy: 0.6397 - lr: 0.0010
Epoch 2/10
1058/1058 [==============================] - ETA: 0s - loss: 1.0349 - accuracy: 0.6547
Epoch 00002: val_loss improved from 1.04322 to 0.56147, saving model to fruits.h5
1058/1058 [==============================] - 276s 261ms/step - loss: 1.0349 - accuracy: 0.6547 - val_loss: 0.5615 - val_accuracy: 0.8253 - lr: 0.0010
Epoch 3/10
1058/1058 [==============================] - ETA: 0s - loss: 0.6329 - accuracy: 0.7907
Epoch 00003: val_loss improved from 0.56147 to 0.38899, saving model to fruits.h5
1058/1058 [==============================] - 266s 251ms/step - loss: 0.6329 - accuracy: 0.7907 - val_loss: 0.3890 - val_accuracy: 0.8706 - lr: 0.0010
Epoch 4/10
105